In [1]:
import torch 
import torch.nn as nn

In [2]:
GPT_CONFIG_350M = {
    "vocab_size":50257,
    "context_length":1024,
    "embedding_dim":1600,
    "num_of_heads":16,
    "num_of_layers":24,
    "drop_rate":0.1,
    "qkv_bias":False
}

In [3]:

class MultiHeadAttention(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,num_of_head,qkv_bias=False):
        super().__init__()
        
        assert (d_out % num_of_head == 0), "d_out must be divisible by num_of_head"
        
        self.d_out = d_out                                   
        self.dropout = nn.Dropout(dropout)
        self.num_of_head = num_of_head                         
        
        self.head_dim = d_out // num_of_head                
        self.out_proj = nn.Linear(d_out, d_out)             
        
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length,context_length), diagonal=1)
        )
        
    def forward(self, x):
        num_of_inputs, num_of_tokens, embedding_dim = x.shape           
        
        queries = self.W_query(x)                            
        keys = self.W_key(x)
        values = self.W_value(x)
        
        queries = queries.view(num_of_inputs, num_of_tokens, self.num_of_head, self.head_dim)      
        keys = keys.view(num_of_inputs, num_of_tokens, self.num_of_head, self.head_dim)       
        values = values.view(num_of_inputs, num_of_tokens, self.num_of_head, self.head_dim)       
        
        
        queries = queries.transpose(1,2)
        keys = keys.transpose(1,2)
        values = values.transpose(1,2)
        
        attention_score = queries @ keys.transpose(2,3)
        mask_bool = self.mask.bool() [:num_of_tokens, :num_of_tokens]
        
        attention_score.masked_fill_(mask_bool, -torch.inf)
        
        k_d = keys.shape[-1]
        attention_weight = torch.softmax(attention_score / k_d**0.5, dim=-1)
        attention_weight = self.dropout(attention_weight)
        
        context_vector = (attention_weight @ values).transpose(1,2)
        
        context_vector = context_vector.contiguous().view(
            num_of_inputs, num_of_tokens, self.d_out
        )
        context_vector = self.out_proj(context_vector)
        return context_vector

In [4]:
class LayerNormalization(nn.Module):
    def __init__(self, embedding_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(embedding_dim))
        self.shift = nn.Parameter(torch.zeros(embedding_dim))
        
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        variance = x.var(dim=-1, keepdim=True)
        
        x_normalized = (x - mean)/ torch.sqrt(variance + self.eps)
        return self.scale * x_normalized + self.shift

In [5]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self,x):
        return 0.5 * x * (
            1 + torch.tanh(torch.sqrt(torch.tensor(2.0/torch.pi)) * (x + 0.44715 * torch.pow(x,3)))
        )

In [6]:
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(config['embedding_dim'], 4* config['embedding_dim']),
            GELU(),
            nn.Linear(4*config['embedding_dim'],config['embedding_dim'])
        )
        
    def forward(self, x):
        return self.layers(x)

In [7]:
class TransformerBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layer_norm1 = LayerNormalization(config['embedding_dim'])
        self.attention = MultiHeadAttention(
            d_in=config['embedding_dim'],
            d_out=config['embedding_dim'],
            context_length=config['context_length'],
            dropout=config['drop_rate'],
            num_of_head=config['num_of_heads'],
            qkv_bias=config['qkv_bias']
        )
        self.dropout = nn.Dropout(config['drop_rate'])
        self.layer_norm2 = LayerNormalization(config['embedding_dim'])     
        self.ff = FeedForward(config)
        
    def forward(self,x):
        shortcut = x
        
        x = self.layer_norm1(x)
        x = self.attention(x)
        x = self.dropout(x)
        
        x = x + shortcut
        
        shortcut = x 
        x = self.layer_norm2(x)
        x = self.ff(x)
        x = self.dropout(x)
        
        x = x + shortcut
        return x

In [8]:
transformerBlock = TransformerBlock(GPT_CONFIG_350M) 

In [9]:
transformerBlock

TransformerBlock(
  (layer_norm1): LayerNormalization()
  (attention): MultiHeadAttention(
    (dropout): Dropout(p=0.1, inplace=False)
    (out_proj): Linear(in_features=1600, out_features=1600, bias=True)
    (W_query): Linear(in_features=1600, out_features=1600, bias=False)
    (W_key): Linear(in_features=1600, out_features=1600, bias=False)
    (W_value): Linear(in_features=1600, out_features=1600, bias=False)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (layer_norm2): LayerNormalization()
  (ff): FeedForward(
    (layers): Sequential(
      (0): Linear(in_features=1600, out_features=6400, bias=True)
      (1): GELU()
      (2): Linear(in_features=6400, out_features=1600, bias=True)
    )
  )
)

In [10]:
class GPTModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        
        self.token_embedding = nn.Embedding(config['vocab_size'], config['embedding_dim'])
        self.positional_embedding = nn.Embedding(config['context_length'], config['embedding_dim'])
        self.dropout_embedding = nn.Dropout(config['drop_rate'])
        
        self.transformer_blocks = nn.Sequential(
            *[TransformerBlock(config) for _ in range(config['num_of_layers'])]
        )
        
        self.final_normalization = LayerNormalization(config['embedding_dim'])
        
        self.output_head = nn.Linear(config['embedding_dim'], config['vocab_size'], bias=False)
        
        
    def forward(self,inputs):
        batch_size, no_of_token = inputs.shape
        token_embedding = self.token_embedding(inputs)
        positional_embedding = self.positional_embedding(
            torch.arange(no_of_token, device=inputs.device)
        )
        
        x = token_embedding + positional_embedding
        x = self.dropout_embedding(x)
        x = self.transformer_blocks(x)
        x = self.final_normalization(x)
        logits = self.output_head(x)
        return logits
    
        

In [11]:
import tiktoken

tokenizer = tiktoken.get_encoding('gpt2')

batch = []

tx1 = "Every efforts moves you"
tx2 = "Everyday hold a"

batch.append(torch.tensor(tokenizer.encode(tx1)))
batch.append(torch.tensor(tokenizer.encode(tx2)))
batch = torch.stack(batch, dim=0)
batch

tensor([[6109, 4040, 6100,  345],
        [6109,  820, 1745,  257]])

In [12]:
batch.shape

torch.Size([2, 4])

In [13]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_350M)
logits = model(batch)
logits

tensor([[[-0.6579,  0.8784,  0.8806,  ...,  0.9194,  0.1150, -0.0935],
         [ 0.8075, -1.2021, -0.8325,  ..., -0.5717,  0.1536,  0.1127],
         [-0.1149, -0.7090, -0.3095,  ...,  0.0235,  0.1909, -0.9879],
         [-0.1309,  0.1216, -0.5488,  ...,  1.1006,  0.2202,  0.1228]],

        [[ 0.3100,  0.2314,  0.1667,  ...,  0.9612,  0.6297,  0.1075],
         [-0.8291, -0.6832, -0.7368,  ...,  0.4337,  0.6186, -0.3144],
         [ 0.0688,  0.0266, -0.6549,  ...,  0.7712, -0.5188,  0.3529],
         [-0.5701, -0.1891, -0.7100,  ...,  0.7228, -0.6595,  0.3389]]],
       grad_fn=<UnsafeViewBackward0>)

In [14]:
logits.shape

torch.Size([2, 4, 50257])